# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import deque
from dqn_agent import Agent
import torch
import json
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)
print('States have shape:', state.shape)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37
States have shape: (37,)


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

In [6]:
def run_episode(train, agent, eps=0):
    """
    Run an episode
    
    Params
    ======
        eps (float): epsilon for the epsilon greedy action selection
    """
    env_info = env.reset(train_mode=train)[brain_name] # reset the environment
    state = env_info.vector_observations[0]  # the state of the environment
    score = 0
    while  True:
        action = agent.act(state, eps)                 # get the action from the agent
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished

        agent.step(state, action, reward, next_state, done)

        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:
            break 
        if train == False :
            time.sleep(0.02) # on my hardware, it goes to fast too clearly see action changes
    return score

In [7]:
# find the best combination of hyper-parameters 
hyper_parameters = {
        'eps_decay': [0.99, 0.995, 0.999],
        'buffer_size':[int(1e5), int(1e6)],
        'batch_size':[64, 256],
        'gamma':[0.9, 0.99, 0.995],
        'tau':[5e-2, 1e-3, 5e-4],
        'update_every' : [2, 4, 8],
        'layers_units' : [[64, 64], [64, 128, 64], [125, 256, 256, 128]]
    }

default = {
        'eps_decay': 0.995,
        'buffer_size':int(1e5),
        'batch_size':64,
        'gamma':0.99,
        'tau':1e-3, 
        'update_every' : 4,
        'layers_units' : [64, 64]
    }

def to_name(parameters) :
    return ' '.join('{}={}'.format(name, str(value)) for name, value in parameters.items())

to_name(default)


'eps_decay=0.995 buffer_size=100000 batch_size=64 gamma=0.99 tau=0.001 update_every=4 layers_units=[64, 64]'

In [8]:
def dqn(parameters, n_episodes=10, save=False):
    
    """Deep Q-Learning.
    
    Params
    ======
        n_episodas (int): maximum number of training episodes
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    lr =5e-4
    agent = Agent(
        state_size,
        action_size,
        seed=0,
        buffer_size=parameters['buffer_size'],
        batch_size=parameters['batch_size'],
        gamma=parameters['gamma'],
        tau=parameters['tau'],
        lr=lr,
        update_every=parameters['update_every'],
        layers_units=parameters['layers_units']
    )
    eps_end = 0.01
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = 1.0                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        score = run_episode(True, agent, eps)    # run the epsiode
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, parameters['eps_decay'] *eps) # decrease epsilon
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        mean_score = np.mean(scores_window)
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, mean_score))
            if save :
                torch.save(agent.qnetwork_local.state_dict(), 'checkpoint_{}.pth'.format(i_episode))
    return scores, mean_score



In [ ]:


# find sequentially the "best" value for each parameters after 1000 episodes. 
# When a parameter is choosen, the others take values from the dqn project
results = {}
for name, parameter in hyper_parameters.items() :
    
    for value in parameter :
        print("\n\nTuning parameter {}, value = {}".format(name, value))
        parameters = default.copy()
        parameters[name] = value
        scores, mean_score = dqn(parameters, 5000)
        results['{}_{}'.format(name, str(value))] = mean_score
        with open('hyper_parameters_tuning_partial.json', 'w') as fp:
            json.dump(results, fp)       

with open('hyper_parameters_tuning.json', 'w') as fp:
    json.dump(results, fp)       



Tuning parameter eps_decay, value = 0.99
Episode 100	Average Score: 0.82
Episode 200	Average Score: 3.61
Episode 300	Average Score: 5.39
Episode 400	Average Score: 5.49
Episode 500	Average Score: 5.38
Episode 600	Average Score: 6.28
Episode 700	Average Score: 7.55
Episode 800	Average Score: 7.38
Episode 803	Average Score: 7.42

In [ ]:
scores, mean_score = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

When finished, you can close the environment.

In [ ]:

agent.qnetwork_local.load_state_dict(torch.load('checkpoint_7800.pth'))
run_episode(False)

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```